In [1]:
import tensorflow as tf 
from tensorflow.keras import layers,Sequential

## 以下是ResNet的基本层的实现, 但是ResNet的基本组成单元block是几个这样的层组成的一个实体

In [6]:
class BasicBlock(layers.Layer):
    def __init__(self, filter_num,stride = 1):
        self.conv1 = layers.Conv2D(filter_num,(3,3),strides=strider,padding='SAME')
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        self.conv2 = layers.Conv2D(filter_num,(3,3),strides=1,padding='SAME')
        self.bn2 = layers.BatchNormalization()
        
        # 这里如果设置stride不风雨1,则需要对短接层也进行一个stride间隔采样卷积
        if strider != 1:
            self.downsample = Sequential()
            self.downsample.add(layers.Conv2D(filter_num,(1,1),strides = stride))
            self.downsample.add(layers.BatchNormalization())
        else:
            self.downsample = lambda x:x  # 如果不间隔采样, 则这个短阶层直接就是输入什么输出什么
        
        self.stride = stride
    
    def __call__(self,inputs,training = None):
        residual = self.downsample(inputs)
        conv1 = self.conv1(inputs)
        bn1 = self.bn1(conv1)
        relu1 = self.relu(bn1)
        conv2 = self.conv2(relu1)
        bn2 = self.bn2(conv2)
        
        add = layers.add([bn2,residual])# 这里的相加注意是layers中的相加
        out = self.relu(add)
        return out 

## ResNet的基本组成单元

In [7]:
def _build_block(block, filter_num, blocks, strides = 1):
    res_blocks = Sequential()
    res_blocks.add(block(filter_num,stride))
    
    for i in range(1,blocks):
        res_blocks.add(block(filter_num,1))
    
    return res_blocks

# ResNet中的一层只会和前面的邻接的一层有短接(接触), 根据这个理念, 我们创新出了DenseNet, 中间的一层可以和前面的所有层都能短接,这种网络中连接更多, 通道数会不断的堆叠不断的增加.